The goal is to study backported library updates in NPM. 

Since not all packages are of interest for such a study, we will select a subset of *required* packages to be considered (the active, sufficiently required ones) and a subset of *dependent* packages (the active ones, depending on a selected one).

In [1]:
import pandas
import tqdm
import portion as P

from version import Version
from parsers import NPMParser, parse_or_empty

%matplotlib inline

In [2]:
MIN_REQUIRED = 5
MIN_ACTIVE = pandas.to_datetime('2019-01-01')

In [3]:
df_releases = (
    pandas.read_csv('../data/NPM-releases.csv.gz')
    .assign(date=lambda d: pandas.to_datetime(d['date'], infer_datetime_format=True))
)

In [4]:
df_dependencies = (
    pandas.read_csv('../data/NPM-dependencies.csv.gz')
)

Let's look at these dataframes:

In [5]:
df_releases.head()

,package,version,major,minor,patch,rank,date,rank_date
0,0,0.0.0,0.0,0.0,0.0,1,2014-04-01 22:51:11,1
1,0-,0.0.1,0.0,0.0,1.0,1,2017-03-06 11:57:07,1
2,0----,1.0.0,1.0,0.0,0.0,1,2017-10-29 23:55:34,1
3,0-0,1.0.2,1.0,0.0,2.0,1,2016-12-04 05:52:10,1
4,0-1-project,0.0.1,0.0,0.0,1.0,1,2018-12-10 06:44:08,1


In [6]:
df_dependencies.head()

,source,version,target,constraint,rank,interval,i_empty,i_major,i_minor,i_patch,i_dev
0,01,0.0.1,commander,1.0.x,1,"[1.0.0,1.1.0)",False,False,False,True,False
1,0815,0.1.0,commander,>= 1.1.1,1,"[1.1.1,+inf)",False,True,True,True,False
2,0815,0.1.0,glob,>= 3.1.17,1,"[3.1.17,+inf)",False,True,True,True,False
3,0815,0.1.0,async,>= 0.1.22,1,"[0.1.22,+inf)",False,True,True,True,False
4,0815,0.1.0,wrench,>= 1.4.4,1,"[1.4.4,+inf)",False,True,True,True,False


Let's ignore inactive packages: 

In [7]:
print('All packages:', len(df_releases.drop_duplicates('package')))
print('Their releases:', len(df_releases))

print()

packages = (
    df_releases
    .sort_values('rank_date')
    .drop_duplicates('package', keep='last')
    [lambda d: d['date'] >= MIN_ACTIVE]
    .package
)
df_releases = df_releases[lambda d: d['package'].isin(packages)]

print('Active packages:', len(df_releases.drop_duplicates('package')))
print('Their releases:', len(df_releases))

All packages: 1217677
Their releases: 9383406

Active packages: 430976
Their releases: 5050117


Let's keep the sufficiently required ones. We only consider dependencies in the latest available version of a package.

In [8]:
df_dependents = (
    df_dependencies
    # Keep only selected packages
    [lambda d: d['source'].isin(packages) & d['target'].isin(packages)]
    # Keep only last release of each package
    .merge(
        (
            df_releases
            [['package', 'rank', 'rank_date']]
            .sort_values(['package', 'rank_date'])
            .drop_duplicates('package', keep='last')
        ),
        how='inner',
        left_on=['source', 'rank'],
        right_on=['package', 'rank'],
    )
    .drop_duplicates(['source', 'target'])  # Effectless
)

print('Required packages:', len(df_dependents.drop_duplicates('target')))
print('At least', MIN_REQUIRED, 'dependents:', len(
    df_dependents
    .groupby('target', sort=False)
    .agg({'source': 'count'})
    [lambda d: d['source'] >= MIN_REQUIRED]
))

Required packages: 78395
At least 5 dependents: 15644


In [9]:
required = (
    df_dependents
    .groupby('target', sort=False)
    .agg({'source': 'count'})
    .reset_index()
    [lambda d: d['source'] >= MIN_REQUIRED]
    .target
)

df_dependents = (
    df_dependents
    [lambda d: d['target'].isin(required)]
)

In [10]:
print('Number of distinct required packages:', len(required))
print('Number of distinct dependent packages:', len(df_dependents.drop_duplicates('source')))
print('Number of dependencies:', len(df_dependents))

Number of distinct required packages: 15644
Number of distinct dependent packages: 247408
Number of dependencies: 1069343


We'll study backported updates for each required packages. Let's identify such backported updates. 
A backported update is an update deployed on a previous major release, assuming semver. 

In [11]:
data = []

for name, group in tqdm.tqdm(df_releases[lambda d: d['package'].isin(required)].groupby('package', sort=False)):
    group = (
        group
        # Identify update kind for each release
        .sort_values('rank')
        .assign(
            kinitial=lambda d: d['major'].shift(1).isnull(),
            kmajor=lambda d: (d['major'] - d['major'].shift(1)).clip(0, 1).astype(bool),
            kminor=lambda d: (d['minor'] - d['minor'].shift(1)).clip(0, 1).astype(bool),
            kpatch=lambda d: (d['patch'] - d['patch'].shift(1)).clip(0, 1).astype(bool),
        )
        .assign(kind=lambda d: d[['kinitial', 'kmajor', 'kminor', 'kpatch']].idxmax(axis=1))
        .replace({'kind': {'kinitial': 'initial', 'kmajor': 'major', 'kminor': 'minor', 'kpatch': 'patch'}})        
        .drop(columns=['kinitial', 'kmajor', 'kminor', 'kpatch'])
        
        # Detect highest major seen so far
        .sort_values(['date', 'rank'])
        .assign(
            highest_major=lambda d: d['major'].expanding().max(),
            highest_rank=lambda d: d['rank'].expanding().max(),
        )
        .assign(
            backported=lambda d: d['highest_rank'].where(d['major'] < d['highest_major'], pandas.np.nan)
        )
        .drop(columns=['highest_major', 'highest_rank'])
        
        # A backported update could have been deployed right before it's "source release" (i.e. the one
        # being backported). We take the closest date to identify from which update a backported one
        # was created. 
        .pipe(lambda df: 
            # Let's find the "previous" release
            df.merge(
                df[['date', 'rank']],
                how='left',
                left_on=['backported'],
                right_on=['rank'],
                suffixes=('', '_previous')
            )
            # Let's find the "next" release
            .merge(
                df[['date', 'rank']].assign(rank=lambda d: d['rank'] - 1),
                how='left',
                left_on=['backported'],
                right_on=['rank'],
                suffixes=('', '_next')
            )
            .assign(rank_next=lambda d: d['rank_next'] + 1)
            # Take closest date
            .assign(backported_from=lambda d:
                d['rank_previous'].where(abs(d['date'] - d['date_previous']) <= abs(d['date'] - d['date_next']), d['rank_next'])
            )
        )
        # Clean unused columns
        .drop(columns=['date_previous', 'date_next', 'rank_previous', 'rank_next'])
        # Booleans for backported
        .assign(backported=lambda d: ~d['backported'].isnull())    
    )
    
    data.append(group)

df_required = (
    pandas.concat(data)
    .sort_values(['package', 'rank'])
    [['package', 'version', 'major', 'minor', 'patch', 'kind', 'rank', 'date', 'rank_date', 'backported', 'backported_from']]
)

100%|██████████| 15644/15644 [09:40<00:00, 26.94it/s]


Let's have a look at the data:

In [14]:
df_required.query('package == "vue-awesome"').iloc[25:35]

,package,version,major,minor,patch,kind,rank,date,rank_date,backported,backported_from
25,vue-awesome,2.3.5,2.0,3.0,5.0,patch,26,2018-03-01 15:51:36,26,False,NaN
28,vue-awesome,2.3.6,2.0,3.0,6.0,patch,27,2018-06-03 03:51:19,29,True,31.0
30,vue-awesome,2.3.7,2.0,3.0,7.0,patch,28,2018-06-03 10:07:02,31,True,32.0
34,vue-awesome,2.3.8,2.0,3.0,8.0,patch,29,2018-07-09 07:15:18,35,True,35.0
26,vue-awesome,3.0.0,3.0,0.0,0.0,major,30,2018-04-02 06:56:11,27,False,NaN
27,vue-awesome,3.0.1,3.0,0.0,1.0,patch,31,2018-06-03 03:39:54,28,False,NaN
29,vue-awesome,3.0.2,3.0,0.0,2.0,patch,32,2018-06-03 10:05:28,30,False,NaN
31,vue-awesome,3.0.3,3.0,0.0,3.0,patch,33,2018-06-27 04:02:24,32,False,NaN
32,vue-awesome,3.0.4,3.0,0.0,4.0,patch,34,2018-07-09 04:05:08,33,False,NaN
33,vue-awesome,3.0.5,3.0,0.0,5.0,patch,35,2018-07-09 07:05:47,34,False,NaN


In [13]:
# Save the world, save data!
df_required.to_csv('../data/required.csv.gz', index=False, compression='gzip')

Let's collect the data for dependent packages.

As a first step, we need to convert dependency constraints to intervals, so we can manipulate constraints and versions.

In [15]:
intervals = dict()
parser = NPMParser()


for constraint in tqdm.tqdm(df_dependents.constraint.drop_duplicates()):
    interval = parse_or_empty(parser, constraint)
    d = {'interval': interval}
    
    if interval.empty:
        d['empty'] = True
        d['major'] = d['minor'] = d['patch'] = d['dev'] = False
    else:
        base = interval.lower 
        d['empty'] = False
        d['major'] = Version(float('inf'), 0, 0) in interval
        d['minor'] = Version(base.major, float('inf'), 0) in interval
        d['patch'] = Version(base.major, base.minor, float('inf')) in interval
        d['dev'] = Version(1, 0, 0) > interval
        
    intervals[constraint] = d
    
# Are all intervals equal to their enclosure? (i.e. are there "gaps"?)
len([i['interval'] for i in intervals.values() if i['interval'] != i['interval'].enclosure])

100%|██████████| 18158/18158 [00:31<00:00, 583.29it/s]


53

Now let's identify the highest accepted version for each constraint. 

In [16]:
data = []

for target, group in tqdm.tqdm(df_dependents.groupby('target', as_index=False, sort=False), leave=True, position=0):
    
    target_releases = (
        df_required[lambda d: d['package'] == target]
        # Convert version to usable objects
        .assign(version=lambda d: d['version'].apply(lambda v: Version(v)))
        # Sort in decreasing order so we can easily find the "highest" accepted version given a constraint
        .sort_values('rank', ascending=False)
    )
    
    # Let's group by constraint, so we do not evaluate a same (target, constraint) twice.
    for constraint, cgroup in group.groupby('constraint', as_index=False, sort=False):
        # Find highest version accepted by a constraint
        d = intervals[constraint]
        interval = d['interval']
        selected = pandas.np.nan
        
        for release in target_releases.itertuples():
            if release.version in interval:
                selected = release.rank
                break  # Because they are sorted by descending rank
            
        data.append((
            cgroup.assign(
                interval=str(interval),
                selected=selected,
                c_empty=d['empty'],
                c_dev=d['dev'],
                c_major=d['major'],
                c_minor=d['minor'],
                c_patch=d['patch'],
            )
        ))
        

100%|██████████| 15644/15644 [16:56<00:00, 15.39it/s] 


Let's combine these data and save them.

In [17]:
df_dependents = (
    pandas.concat(data)
    .sort_values(['source', 'target'])
    [['source', 'version', 'rank', 'target', 'constraint', 'interval', 'selected', 'c_empty', 'c_dev', 'c_major', 'c_minor', 'c_patch']]
)        

In [18]:
df_dependents.to_csv('../data/dependents.csv.gz', index=False, compression='gzip')